In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json
import os
import shutil
import math
import pandas as pd
import cv2
import glob
import numpy as np

In [3]:
# 실행 경로 변경 및 확인
os.chdir('/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data')
os.getcwd()

'/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data'

### files, gloss, skels 생성

* 저장 위치는 새로 만든 `text2keypoint` 공유 드라이브

#### 결측 파일 확인

`키포인트txt`와 `형태소Json` 중 하나만 속한 파일이 있어서 이를 확인했었음!

In [7]:
file_dir = '/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/' # 시작 dir

hwaja_list = ['REAL01', 'REAL02','REAL02_정은word', 'REAL05_정은word', 'REAL06', 'REAL10', 'REAL10_정은'] # 화자 폴더 list
folder_list = ['키포인트Txt', '형태소Json'] # 화자 폴더 내 필요한 폴더 list

for folder in hwaja_list:
    step1_dir = file_dir + folder
    print(step1_dir)
    
    for dir2 in folder_list:
        step2_dir = step1_dir + '/' + dir2
        print(step2_dir)

/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/키포인트Txt
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/형태소Json
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL02
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL02/키포인트Txt
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL02/형태소Json
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL02_정은word
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL02_정은word/키포인트Txt
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL02_정은word/형태소Json
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL05_정은word
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL05_정은word/키포인트Txt
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL05_정은word/형태소Json
/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL06
/content/d

In [8]:
txt_files = []
json_files = []

# 'REAL01', 'REAL02','REAL02_정은word', 'REAL05_정은word', 'REAL06', 'REAL10', 'REAL10_정은'
for folder in hwaja_list:
    step1_dir = file_dir + folder
    # print(step1_dir)

    # '키포인트Txt', '형태소Json'
    for dir2 in folder_list:
        step2_dir = step1_dir + '/' + dir2 + '/'
        # print(step2_dir)

        if dir2 == '키포인트Txt': # txt_files에 파일명 추가
            # print(glob.glob(os.path.join(step2_dir,'**','*.txt'), recursive=True))
            txt_file = glob.glob(os.path.join(step2_dir,'**','*.txt'), recursive=True)
            txt_files.append(txt_file)

        elif dir2 == '형태소Json':# json_files에 파일명 추가
            # print(glob.glob(os.path.join(step2_dir,'**','*.json'), recursive=True))
            json_file = glob.glob(os.path.join(step2_dir,'**','*.json'), recursive=True)
            json_files.append(json_file)


txt_files = [name for json_file in txt_files for name in json_file] # 1차원으로 변경
txt_files.sort() # 파일명 순으로 정렬 (**무조건 정렬해줘야 함!)

json_files = [name for json_file in json_files for name in json_file]
json_files.sort()

print(len(txt_files))
print(len(json_files))

1456
1458


키포인트와 형태소 파일의 개수가 같으면 밑에 코드 진행
-> 근데 다르니까 확인해보자,,

In [10]:
# 이름 부분만 가져와서 비교
mor = [json_file.split('/')[8].split('_morpheme')[0] for json_file in json_files]
key = [txt_file.split('/')[8].split('.txt')[0] for txt_file in txt_files]

print(mor[:5])
print(key[:5])

print('키포인트Txt가 없는 것 : ', set(mor)-set(key))
print('형태소Json이 없는 것 : ', set(key)-set(mor))

['NIA_SL_SEN0282_REAL01_F', 'NIA_SL_SEN0283_REAL01_F', 'NIA_SL_SEN0284_REAL01_F', 'NIA_SL_SEN0285_REAL01_F', 'NIA_SL_SEN0286_REAL01_F']
['NIA_SL_SEN0282_REAL01_F', 'NIA_SL_SEN0283_REAL01_F', 'NIA_SL_SEN0284_REAL01_F', 'NIA_SL_SEN0285_REAL01_F', 'NIA_SL_SEN0286_REAL01_F']
키포인트Txt가 없는 것 :  {'NIA_SL_WORD0110_REAL05_F', 'NIA_SL_SEN0004_REAL10_F'}
형태소Json이 없는 것 :  set()


해당되는 형태소Json은 json_files에서 제거하고 진행

In [11]:
for i in set(mor)-set(key): # 해당 값이 있으면 제거
    # print(i)
    json_files = [json_file for json_file in json_files if json_file.find(i) == -1]

txt_files.sort()
json_files.sort()

print(len(txt_files))
print(len(json_files))

1456
1456


gloss 이상값도 확인

In [20]:
### gloss_list

gloss_list = [] # gloss 저장 리스트

for json_name in json_files: # 위에서 만든 json_files 그대로 사용,
    text_list = []

    with open(json_name, "r", encoding='UTF-8') as json_file:
        json_data = json.load(json_file)
        # print(json_data)

    # text_list.append(json_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name'] # gloss가 있는 위치
        text_list.append(text)

    gloss_list.append(text_list) # gloss list에 추가

print(len(gloss_list))
gloss_list[:5]

1456


[['지하철', '가방', '두다', '잊다'],
 ['지하철', '반대', '건너', '가다'],
 ['지하철', '반대', '건너', '어떻게'],
 ['아이', '귀신', '찾다', '도움'],
 ['아이', '귀신', '찾다', '도움', '할수있다']]

In [21]:
dic = dict(zip(json_files, gloss_list))
len(dic)

1456

In [23]:
# 결측값 확인
# sorted(dic.items(), key=lambda x:x[1]) 

결측값 확인 결과 다음과 같음, json 파일 열어서 확인해보기

In [28]:
path = '/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/'
null_list = ['REAL02/형태소Json/NIA_SL_WORD0974_REAL02_F_morpheme.json', 'REAL02/형태소Json/NIA_SL_WORD1097_REAL02_F_morpheme.json',
             'REAL02/형태소Json/NIA_SL_WORD1098_REAL02_F_morpheme.json', 'REAL02/형태소Json/NIA_SL_WORD0928_REAL02_F_morpheme.json']

for i in range(len(null_list)):
    with open(path + null_list[i], encoding='UTF-8') as json_file: #손글씨체 이미지
        data = json.load(json_file)
    print(null_list[i].split('/')[2], ':', data)

NIA_SL_WORD0974_REAL02_F_morpheme.json : {'metaData': {'url': 'https://blackolivevideo.blob.core.windows.net/sign-language/0928_moyu113/NIA_SL_WORD0974_REAL02_F.mp4', 'name': 'NIA_SL_WORD0974_REAL02_F.mp4', 'duration': 4.084, 'exportedOn': '2020/10/06'}, 'data': []}
NIA_SL_WORD1097_REAL02_F_morpheme.json : {'metaData': {'url': 'https://blackolivevideo.blob.core.windows.net/sign-language/0928_moyu125/NIA_SL_WORD1097_REAL02_F.mp4', 'name': 'NIA_SL_WORD1097_REAL02_F.mp4', 'duration': 3.484, 'exportedOn': '2020/10/06'}, 'data': []}
NIA_SL_WORD1098_REAL02_F_morpheme.json : {'metaData': {'url': 'https://blackolivevideo.blob.core.windows.net/sign-language/0928_moyu125/NIA_SL_WORD1098_REAL02_F.mp4', 'name': 'NIA_SL_WORD1098_REAL02_F.mp4', 'duration': 2.717, 'exportedOn': '2020/10/06'}, 'data': []}
NIA_SL_WORD0928_REAL02_F_morpheme.json : {'metaData': {'url': 'https://blackolivevideo.blob.core.windows.net/sign-language/0928_sjim123/NIA_SL_WORD0928_REAL02_F.mp4', 'name': 'NIA_SL_WORD0928_REAL02_

해당 Json 파일은 형태소Json과 키포인트txt에서 모두 제거하고 진행

In [30]:
null_list = ['NIA_SL_WORD0974_REAL02_F', 'NIA_SL_WORD1097_REAL02_F', 'NIA_SL_WORD1098_REAL02_F', 'NIA_SL_WORD0928_REAL02_F']

for i in null_list: # 해당 값이 있으면 제거
    # print(i)
    json_files = [json_file for json_file in json_files if json_file.find(i) == -1]
    txt_files = [txt_file for txt_file in txt_files if txt_file.find(i) == -1]

txt_files.sort()
json_files.sort()

print(len(txt_files))
print(len(json_files))

1452
1452


**최종 사용하는 데이터셋 개수 1452개**

#### trail.files 생성

`files`는 형태소Json의 파일명을 가짐

In [33]:
# 위에서 만든 파일
print(len(json_files))
json_files[:5]

1452


['/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/형태소Json/NIA_SL_SEN0282_REAL01_F_morpheme.json',
 '/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/형태소Json/NIA_SL_SEN0283_REAL01_F_morpheme.json',
 '/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/형태소Json/NIA_SL_SEN0284_REAL01_F_morpheme.json',
 '/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/형태소Json/NIA_SL_SEN0285_REAL01_F_morpheme.json',
 '/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/형태소Json/NIA_SL_SEN0286_REAL01_F_morpheme.json']

In [34]:
filename_list = [] # 이름 list

for i in range(len(json_files)):
    filename = json_files[i].split('/')[8].split('_morpheme')[0] # _morpheme.json 부분 제거
    filename_list.append(filename)

print(len(filename_list))
filename_list[:5]

1452


['NIA_SL_SEN0282_REAL01_F',
 'NIA_SL_SEN0283_REAL01_F',
 'NIA_SL_SEN0284_REAL01_F',
 'NIA_SL_SEN0285_REAL01_F',
 'NIA_SL_SEN0286_REAL01_F']

In [15]:
# filename_list # 문제 없음

In [35]:
# trial.files 
file = '/content/drive/Shareddrives/text2keypoint/data/trial.files' # 저장할 경로 + 파일명

with open(file, 'w') as f:
    for text in filename_list:
        f.writelines(text) # 한줄씩 적기
        f.write("\n") # enter 추가

In [36]:
# check
r_file = file = "/content/drive/Shareddrives/text2keypoint/data/trial.files"
with open(r_file, 'r') as fr:
    data = fr.readlines()
print(data[:5])

['NIA_SL_SEN0282_REAL01_F\n', 'NIA_SL_SEN0283_REAL01_F\n', 'NIA_SL_SEN0284_REAL01_F\n', 'NIA_SL_SEN0285_REAL01_F\n', 'NIA_SL_SEN0286_REAL01_F\n']


In [19]:
# data # 문제 없음

#### trial.gloss 생성

`gloss`는 형태소Json의 gloss(수어소, 형태소)를 가짐

In [37]:
### gloss_list

gloss_list = [] # gloss 저장 리스트

for json_name in json_files: # 위에서 만든 json_files 그대로 사용,
    text_list = []

    with open(json_name, "r", encoding='UTF-8') as json_file:
        json_data = json.load(json_file)
        # print(json_data)

    # text_list.append(json_name)
    for j in range(len(json_data["data"])):
        text = json_data["data"][j]["attributes"][0]['name'] # gloss가 있는 위치
        text_list.append(text)

    gloss_list.append(text_list) # gloss list에 추가

print(len(gloss_list))
gloss_list[:5]

1452


[['지하철', '가방', '두다', '잊다'],
 ['지하철', '반대', '건너', '가다'],
 ['지하철', '반대', '건너', '어떻게'],
 ['아이', '귀신', '찾다', '도움'],
 ['아이', '귀신', '찾다', '도움', '할수있다']]

In [38]:
# trial.gloss(src)  #형태소 txt 파일 생성
file = '/content/drive/Shareddrives/text2keypoint/data/trial.gloss'

with open(file, 'w') as f:
    for i in range(len(gloss_list)):
        for j in gloss_list[i]:
            if "(" in j: # 괄호() 있으면 제거
                rem = j.split("(")
                f.write(rem[0]+" ")
                if j == gloss_list[i][-1]:
                    f.write(rem[1].rstrip(")"))
                else:
                    f.write(rem[1].rstrip(")")+" ")
            else:
                if j == gloss_list[i][-1]:
                    f.write(j)
                else:
                    f.write(j+" ")

        f.write("\n")

In [39]:
# check
r_file = file = "/content/drive/Shareddrives/text2keypoint/data/trial.gloss"

with open(r_file, 'r') as fr:
    data = fr.readlines()

print(data[:5])

['지하철 가방 두다 잊다\n', '지하철 반대 건너 가다\n', '지하철 반대 건너 어떻게\n', '아이 귀신 찾다 도움\n', '아이 귀신 찾다 도움 할수있다\n']


In [41]:
# data # 문제 없음

#### trial.skels 생성

`skels`는 키포인트txt의 키포인트 값을 가짐

* one-hot vector 제거

* CE값 추가

In [42]:
# 위에서 만든 파일 사용 
print(len(txt_files)) # (아직 키포인트txt가 없어서 0개)
txt_files[:5]

1452


['/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/키포인트Txt/NIA_SL_SEN0282_REAL01_F.txt',
 '/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/키포인트Txt/NIA_SL_SEN0283_REAL01_F.txt',
 '/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/키포인트Txt/NIA_SL_SEN0284_REAL01_F.txt',
 '/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/키포인트Txt/NIA_SL_SEN0285_REAL01_F.txt',
 '/content/drive/Shareddrives/투빅스_컨퍼런스_수어/new_data/REAL01/키포인트Txt/NIA_SL_SEN0286_REAL01_F.txt']

In [32]:
with open(txt_files[0], 'r') as fr:
    data = fr.readlines() # txt파일 읽어오기

    for i in range(len(data)):
        each = data[i].split(",") # ,로 나누고
        each[-1] = each[-1].rstrip("\n") # \n 제거
        each = [line.strip(" ") for line in each]
        each = " ".join(each[1:])
        
        each = each[:-9] # one-hot vector 삭제

        # CE값 추가 (CE값 = 현재 시퀀스의 위치 / 시퀀스의 길이) 0에서 시작해서 1로 끝나도록
        each = each + str(round(1/(len(data)-1)*i,4)) + ' ' 
        
        if i == len(data)-1: # 마지막이 아니면,
            each = each[:-1] # 맨오른쪽 띄어쓰기 제거

        if i==0 or i==len(data)//2 or i==len(data)-1: # 시작, 중간, 끝 check
            print(len(each)//255, each)

10 0.4830508 0.21037327 0.4846753 0.4092283 0.40039796 0.4174271 0.38193262 0.6409375 0.37268066 0.83712846 0.5689697 0.4038012 0.5951318 0.6299722 0.6042578 0.8289427 0.48620704 0.8779948 0.43409082 0.88072914 0.5429297 0.8780035 0.4617129 0.18582118 0.49844727 0.18583593 0.43717775 0.22661287 0.5260449 0.22392535 0.44013673 0.19827604 0.44106397 0.21640365 0.44245458 0.2361797 0.44291797 0.2543073 0.4461626 0.27078733 0.45126075 0.28809115 0.46053076 0.29715538 0.46980077 0.30457118 0.48092481 0.30869097 0.49205077 0.30539498 0.5022461 0.29797918 0.51197755 0.28891492 0.5175391 0.27408332 0.5207861 0.25595573 0.52171385 0.23700349 0.52356446 0.21887587 0.5240283 0.20321961 0.44847998 0.17602777 0.4535781 0.16531597 0.46053076 0.16284375 0.4674834 0.16449219 0.4730454 0.17108421 0.48602295 0.17108421 0.49158692 0.16449219 0.4994629 0.16366841 0.50687987 0.16531597 0.5129053 0.175204 0.47999755 0.18838802 0.47999755 0.1974514 0.47999755 0.20981163 0.47999755 0.21969965 0.47258154 0.236

In [43]:
# trial.skels(trg)

w_file = "/content/drive/Shareddrives/text2keypoint/data/trial.skels"

with open(w_file, 'w') as fw:
    
    for r_file in txt_files: # 키포인트txt 파일 하나씩 불러오기
        with open(r_file, 'r') as fr:
            data = fr.readlines() # txt파일 읽어오기
            
        for i in range(len(data)):
            each = data[i].split(",") # ,로 나누고
            each[-1] = each[-1].rstrip("\n") # \n 제거
            each = [line.strip(" ") for line in each]
            each = " ".join(each[1:])
            
            each = each[:-9] # one-hot vector 삭제

            # CE값 추가 (CE값 = 현재 시퀀스의 위치 / 시퀀스의 길이) 0에서 시작해서 1로 끝나도록
            each = each + str(round(1/(len(data)-1)*i,4)) + ' ' 
            
            if i == len(data)-1: # 마지막이 아니면,
                each = each[:-1] # 맨오른쪽 띄어쓰기 제거

            fw.write(''.join(each))
        fw.write("\n")

In [45]:
# check
r_file = file = "/content/drive/Shareddrives/text2keypoint/data/trial.skels"

with open(r_file, 'r') as fr:
    data = fr.readlines()

print(data[:5])

['0.4830508 0.21037327 0.4846753 0.4092283 0.40039796 0.4174271 0.38193262 0.6409375 0.37268066 0.83712846 0.5689697 0.4038012 0.5951318 0.6299722 0.6042578 0.8289427 0.48620704 0.8779948 0.43409082 0.88072914 0.5429297 0.8780035 0.4617129 0.18582118 0.49844727 0.18583593 0.43717775 0.22661287 0.5260449 0.22392535 0.44013673 0.19827604 0.44106397 0.21640365 0.44245458 0.2361797 0.44291797 0.2543073 0.4461626 0.27078733 0.45126075 0.28809115 0.46053076 0.29715538 0.46980077 0.30457118 0.48092481 0.30869097 0.49205077 0.30539498 0.5022461 0.29797918 0.51197755 0.28891492 0.5175391 0.27408332 0.5207861 0.25595573 0.52171385 0.23700349 0.52356446 0.21887587 0.5240283 0.20321961 0.44847998 0.17602777 0.4535781 0.16531597 0.46053076 0.16284375 0.4674834 0.16449219 0.4730454 0.17108421 0.48602295 0.17108421 0.49158692 0.16449219 0.4994629 0.16366841 0.50687987 0.16531597 0.5129053 0.175204 0.47999755 0.18838802 0.47999755 0.1974514 0.47999755 0.20981163 0.47999755 0.21969965 0.47258154 0.2361

In [52]:
type(data[0])

str

#### train / test / dev : files, gloss, skels 생성

만들어진 trial.files, trial.gloss, trial.skels를 각각

train / test / dev로 나누어 생성


* train set : 최대한 중복이 없는 애들로 구성
* test set : 5개만 가져오기
* val set : 중복인 애들로 구성

In [4]:
# 생성된 파일 불러오기
files_file = file = "/content/drive/Shareddrives/text2keypoint/data/trial.files"
gloss_file = file = "/content/drive/Shareddrives/text2keypoint/data/trial.gloss"
skels_file = file = "/content/drive/Shareddrives/text2keypoint/data/trial.skels"

with open(files_file, 'r') as fr:
    files = fr.readlines()
    
with open(gloss_file, 'r') as fr:
    gloss = fr.readlines()

with open(skels_file, 'r') as fr:
    skels = fr.readlines()
    
print(len(files))
print(len(gloss))
print(len(skels))

1452
1452
1452


In [5]:
# 불러온 files, gloss, skels를 df에 추가
df = pd.DataFrame({'files':files, 'gloss':gloss, 'skels':skels})
df['files'] = df['files'].str.replace('\n','')
df['gloss'] = df['gloss'].str.replace('\n','')
df['skels'] = df['skels'].str.replace('\n','')

In [6]:
print(df.shape)
df[:3]

(1452, 3)


,files,gloss,skels
0,NIA_SL_SEN0282_REAL01_F,지하철 가방 두다 잊다,0.4830508 0.21037327 0.4846753 0.4092283 0.400...
1,NIA_SL_SEN0283_REAL01_F,지하철 반대 건너 가다,0.48313525 0.21303038 0.48462695 0.4036875 0.4...
2,NIA_SL_SEN0284_REAL01_F,지하철 반대 건너 어떻게,0.4800464 0.22387934 0.4846631 0.39825693 0.40...


아래 코드는 데이터 다 만들어지면, 비율 확인해보면서 수정할 예정

비율 9:1에 가깝게 맞춤

In [8]:
#### 최대한 랜덤하게 돌리기 위해 df 행 섞기
df = df.sample(frac=1).reset_index(drop=True)

#### 중복 추출
df_train = df.drop_duplicates(['gloss']) # train
df_dup = df[df.duplicated(['gloss'])] # 중복 -> test / val

df_dev = df_dup.drop_duplicates(['gloss']) # val
df_test = df_dup[df_dup.duplicated(['gloss'])][:5] # test : 5개만
df_etc = df_dup[df_dup.duplicated(['gloss'])][5:] # ratio 조정

# size
print('df:',df.shape)
print('df_train:',df_train.shape)
print('df_dup:',df_dup.shape)
print('df_dev:',df_dev.shape)
print('df_test:',df_test.shape)
print('df_etc:',df_etc.shape)
print(len(df_train), len(df_dev)*9)

df_train = pd.concat([df_train, df_etc])

print(len(df_train), len(df_dev)*9)

df: (1452, 3)
df_train: (1220, 3)
df_dup: (232, 3)
df_dev: (145, 3)
df_test: (5, 3)
df_etc: (82, 3)
1220 1305
1302 1305


**train : 1302 / test : 5 / dev : 145** 이므로 이대로 진행

#### train.files / test.files / dev.files

In [9]:
# train
file_list = df_train['files'].tolist()
file_train = '\n'.join(file_list)

# test
file_list = df_test['files'].tolist()
file_test = '\n'.join(file_list)

# dev
file_list = df_dev['files'].tolist()
file_dev = '\n'.join(file_list)

In [10]:
# files 생성

# train
w_file = "/content/drive/Shareddrives/text2keypoint/data/train.files" # 저장할 경로 + 파일명

with open(w_file, 'w') as fw:
    fw.write(file_train)
    fw.close()

# dev
w_file = "/content/drive/Shareddrives/text2keypoint/data/test.files"

with open(w_file, 'w') as fw:
    fw.write(file_test)
    fw.close()

# dev
w_file = "/content/drive/Shareddrives/text2keypoint/data/dev.files"

with open(w_file, 'w') as fw:
    fw.write(file_dev)
    fw.close()

In [11]:
# check
r_file = "/content/drive/Shareddrives/text2keypoint/data/train.files"

with open(r_file, 'r') as fr:
    data = fr.readlines()

print(data[:5])

['NIA_SL_WORD1040_REAL02_F\n', 'NIA_SL_SEN0689_REAL10_F\n', 'NIA_SL_WORD0871_REAL02_F\n', 'NIA_SL_WORD0370_REAL02_F\n', 'NIA_SL_SEN0318_REAL01_F\n']


#### trial.gloss / test.gloss / dev.gloss

In [12]:
# train
gloss_list = df_train['gloss'].tolist()
gloss_train = '\n'.join(gloss_list)

# test
gloss_list = df_test['gloss'].tolist()
gloss_test = '\n'.join(gloss_list)
gloss_test[:100]

# dev
gloss_list = df_dev['gloss'].tolist()
gloss_dev = '\n'.join(gloss_list)

In [16]:
# gloss_train # 문제 없음

In [13]:
# files 생성

# train
w_file = "/content/drive/Shareddrives/text2keypoint/data/train.gloss"

with open(w_file, 'w') as fw:
    fw.write(gloss_train)
    fw.close()

# dev
w_file = "/content/drive/Shareddrives/text2keypoint/data/test.gloss"

with open(w_file, 'w') as fw:
    fw.write(gloss_test)
    fw.close()

# dev
w_file = "/content/drive/Shareddrives/text2keypoint/data/dev.gloss"

with open(w_file, 'w') as fw:
    fw.write(gloss_dev)
    fw.close()

In [14]:
# check
r_file = "/content/drive/Shareddrives/text2keypoint/data/dev.gloss"

with open(r_file, 'r') as fr:
    data = fr.readlines()

print(data[:5])

['전 카드\n', '쥐 가다내리다 가다\n', '쥐 가다 가다 내리다\n', '자동차\n', '왼쪽\n']


In [ ]:
# data # 문제 없음

#### train.skels / test.skels / dev.skels

In [17]:
# train
skels_list = df_train['skels'].tolist()
skels_train = '\n'.join(skels_list)

# test
skels_list = df_test['skels'].tolist()
skels_test = '\n'.join(skels_list)

# dev
skels_list = df_dev['skels'].tolist()
skels_dev = '\n'.join(skels_list)

In [18]:
# files 생성

# train
w_file = "/content/drive/Shareddrives/text2keypoint/data/train.skels"

with open(w_file, 'w') as fw:
    fw.write(skels_train)
    fw.close()

# test
w_file = "/content/drive/Shareddrives/text2keypoint/data/test.skels"

with open(w_file, 'w') as fw:
    fw.write(skels_test)
    fw.close()

# dev
w_file = "/content/drive/Shareddrives/text2keypoint/data/dev.skels"

with open(w_file, 'w') as fw:
    fw.write(skels_dev)
    fw.close()

In [19]:
# check
r_file = "/content/drive/Shareddrives/text2keypoint/data/train.skels"

with open(r_file, 'r') as fr:
    data = fr.readlines()

print(data[:2])

['0.48773682 0.23203386 0.48316845 0.39564323 0.40958545 0.3901953 0.34820703 0.5753863 0.40038672 0.44464147 0.5582373 0.39827693 0.61965823 0.5753793 0.5705957 0.44750085 0.4847334 0.861638 0.43714112 0.8616519 0.5337793 0.8616554 0.47249657 0.20471181 0.5077002 0.20753819 0.45551464 0.22386199 0.530669 0.24565019 0.4538911 0.20599133 0.45308983 0.22379687 0.45308983 0.24088976 0.45308983 0.2579835 0.45509326 0.2750764 0.45869872 0.28932118 0.46550927 0.30000436 0.47472364 0.30926302 0.48473924 0.31282464 0.49475586 0.31282464 0.5047705 0.30712673 0.51358396 0.29929167 0.5199951 0.28575957 0.52319825 0.27009115 0.52600586 0.2529974 0.52760744 0.23661631 0.52920896 0.21952344 0.46230468 0.19103386 0.4679131 0.1853368 0.47432324 0.18391234 0.4807334 0.1853368 0.4859414 0.19103386 0.4971582 0.19174653 0.50356936 0.18818575 0.51038086 0.18960936 0.5167871 0.19388281 0.5211963 0.20314236 0.49074706 0.2081276 0.48994628 0.21881077 0.48874512 0.22878212 0.48794433 0.2394653 0.48033252 0.248